### **기본 세팅**
---

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install tqdm
!pip install catboost

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (3,365 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

In [ ]:
import sys
import math

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from matplotlib import font_manager, rc
rc('font', family='NanumBarunGothic') 

data_path = "/content/drive/Shareddrives/Parking_pred/data/Encoded/"
result_path = "/content/drive/Shareddrives/Parking_pred/jihoon/result/"

train = pd.read_excel(data_path + "train.xlsx")
test = pd.read_excel(data_path + "test.xlsx")
api_lh = pd.read_excel(data_path + "openapi_lh.xlsx")
api_mh = pd.read_excel(data_path + "openapi_mh.xlsx")
age_gender = pd.read_excel(data_path + "age_gender_info.xlsx")

submission = pd.read_excel(data_path + "sample_submission.xlsx")

train.shape, test.shape, api_lh.shape, api_mh.shape

((2952, 15), (1022, 14), (5604, 8), (13708, 10))

In [ ]:
api_lh.head()

,단지코드,총세대수,지역,공급유형,전용면적,전용면적별세대수,임대보증금,임대료
0,시흥목감 A-1BL 7단지,402,경기도 시흥시,영구임대,21.97,204,13315000,89840
1,시흥목감 A-5BL 13단지,72,경기도 시흥시,영구임대,26.45,72,13631000,88000
2,시흥목감 B5블록 10년공공임대,325,경기도 시흥시,공공임대,84.77,92,74000000,590000
3,시흥목감 B5블록 10년공공임대,325,경기도 시흥시,공공임대,74.98,150,63000000,560000
4,시흥목감 B5블록 10년공공임대,325,경기도 시흥시,공공임대,74.77,71,63000000,560000


In [ ]:
api_lh['공급유형'].unique()

array(['영구임대', '공공임대', '행복주택', '국민임대', '신축다세대매입임대', '장기전세'], dtype=object)

In [ ]:
api_lh['단지코드'].unique().size

1589

In [ ]:
api_mh.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,50년임대,38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,50년임대,40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,50년임대,50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,50년임대,33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,50년임대,32.7600,534,13510000,178000,263


In [ ]:
api_mh['임대건물구분'].unique()

array(['아파트'], dtype=object)

In [ ]:
api_mh['지역'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [ ]:
api_mh['공급유형'].unique()

array(['50년임대', '장기전세', '국민임대', '매입임대', '행복주택', '영구임대', '10년임대', '5년임대'],
      dtype=object)

In [ ]:
api_mh[api_mh['주차수']!=0].index

KeyError: ignored

In [ ]:
api_mh['단지코드'].unique().size

In [ ]:
api_mh.describe()

In [ ]:
# new_api_mh = pd.DataFrame(columns=api_mh.columns)
# for i, code in tqdm(enumerate(api_mh['단지코드'].unique())):
#     temp = api_mh[api_mh['단지코드']==code].copy()
#     for j, idx in enumerate(temp.index):
#         temp.loc[idx, '총세대수'] = temp['공급유형별세대수'].unique().sum()
#     new_api_mh = new_api_mh.append(temp)

# new_api_mh.reset_index(drop = True, inplace=True)
# new_api_mh.to_excel(data_path + "new_openapi_mh.xlsx")

In [ ]:
train.describe()

In [ ]:
train.info(), train.isna().sum()

In [ ]:
for c in submission['code'].unique():
    if not c in test['단지코드'].unique():
        print(c)

In [ ]:
test.info(), test.isna().sum()

### **오류 데이터 처리**
---

In [ ]:
train_code = ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
test_code = ['C2335', 'C1327']
submission_code = ['C2335', 'C1327', 'C2675']
for c in train_code:
    train = train[train['단지코드'] != c]

# for c in test_code:
#     test = test[test['단지코드'] != c]

# for c in submission_code:
#     submission = submission[submission['code'] != c]

train.shape, test.shape, submission.shape

((2896, 15), (1022, 14), (150, 2))

In [ ]:
c2085 = train[train['단지코드'] == 'C2085'].index
train.loc[c2085, '단지코드'] = 'C1397'

c1397 = train[train['단지코드'] == 'C1397'].index
train.loc[c1397, '총세대수'] = 1339

In [ ]:
c2461 = train[train['단지코드'] == 'C2431'].index
train.loc[c2461, '단지코드'] = 'C1649'

c1649 = train[train['단지코드'] == 'C1649'].index
train.loc[c1649, '총세대수'] = 1047
train.loc[c1649, '등록차량수'] = 1214

In [ ]:
c2675 = test[test['단지코드'] == 'C2675']
c2675['등록차량수'] = 1279
c2675['단지코드'] = 'C1036'
c2675

# test = test[test['단지코드'] != 'C2675']

train = train.append(c2675)
train = train.reset_index(drop=True)
c1036 = train[train['단지코드'] == 'C1036'].index
train.loc[c1036, '총세대수'] = 1254

test[test['단지코드'] == 'C2675'], train[train['단지코드'] == 'C1036']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(      단지코드  총세대수  ... 도보 10분거리 내 버스정류장 수 단지내주차면수
 579  C2675   512  ...                  3    1016
 580  C2675   512  ...                  3    1016
 581  C2675   512  ...                  3    1016
 582  C2675   512  ...                  3    1016
 583  C2675   512  ...                  3    1016
 584  C2675   512  ...                  3    1016
 585  C2675   512  ...                  3    1016
 586  C2675   512  ...                  3    1016
 
 [8 rows x 14 columns],
        단지코드  총세대수 임대건물구분  ... 도보 10분거리 내 버스정류장 수 단지내주차면수  등록차량수
 1490  C1036  1254    아파트  ...                3.0    1016   1279
 1491  C1036  1254    아파트  ...                3.0    1016   1279
 1492  C1036  1254    아파트  ...                3.0    1016   1279
 1493  C1036  1254    아파트  ...                3.0    1016   1279
 1494  C1036  1254    아파트  ...                3.0    1016   1279
 1495  C1036  1254    아파트  ...                3.0    1016   1279
 1496  C1036  1254    아파트  ...                3.0    1016   1279
 289

In [ ]:
def get_mod_miss_households(data):
    total = data.iloc[0]['총세대수']
    sum_hh = data['전용면적별세대수'].sum()
    miss = total - sum_hh
    prop = (data['전용면적별세대수'] / sum_hh).tolist()
    miss_prop = [int(p * miss) for p in prop]
    int_miss = miss - sum(miss_prop)
    for i in range(int_miss):
        miss_prop[(i)%len(miss_prop)] += 1

    return miss_prop


total_miss = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804',
                'C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
                'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
                'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
                'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
                'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']

for tm in total_miss:
    if train[train['단지코드'] == tm].shape[0] == 0:
        print("Empty: {}".format(tm))
        continue
    else:
        mod = get_mod_miss_households(train[train['단지코드'] == tm])
        idx = train[train['단지코드'] == tm].index
        train.loc[idx, '전용면적별세대수'] += mod
        # print(train[train['단지코드'] == tm][["총세대수", "전용면적별세대수"]])

train.shape, test.shape

Empty: C2497
Empty: C1218
Empty: C1894
Empty: C2411
Empty: C1812
Empty: C1030
Empty: C1749
Empty: C1349
Empty: C2043
Empty: C1229
Empty: C2363
Empty: C1414
Empty: C2174
Empty: C2404
Empty: C1683
Empty: C1038
Empty: C2456
Empty: C1266
Empty: C1267
Empty: C2189


((2904, 15), (1022, 14))

In [ ]:
train.reset_index(drop = True, inplace=True)
test.reset_index(drop = True, inplace=True)
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000.0,82940.0,0.0,3.0,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000.0,107130.0,0.0,3.0,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000.0,107130.0,0.0,3.0,624,205
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000.0,149760.0,0.0,3.0,624,205
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000.0,149760.0,0.0,3.0,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,C1036,1254,아파트,경기도,국민임대,46.90,50,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2900,C1036,1254,아파트,경기도,국민임대,46.90,88,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2901,C1036,1254,아파트,경기도,국민임대,47.40,30,9,A,34082000.0,232200.0,0.0,3.0,1016,1279
2902,C1036,1254,아파트,경기도,국민임대,51.93,75,9,A,43819000.0,275900.0,0.0,3.0,1016,1279


In [ ]:
## 지역
api_mh = api_mh[api_mh['지역'] != "인천광역시"]
api_mh

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,50년임대,38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,50년임대,40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,50년임대,50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,50년임대,33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,50년임대,32.7600,534,13510000,178000,263
...,...,...,...,...,...,...,...,...,...,...
13703,30699399,71,아파트,제주특별자치도,10년임대,84.7200,71,0,0,306
13704,30699399,71,아파트,제주특별자치도,10년임대,84.8200,71,0,0,306
13705,30699399,71,아파트,제주특별자치도,10년임대,84.8400,71,0,0,306
13706,30699399,71,아파트,제주특별자치도,10년임대,84.8500,71,0,0,306


In [ ]:
## 공급유형
print(train['공급유형'].unique())
print(api_mh['공급유형'].unique())

idx_5rent = api_mh[api_mh['공급유형'] == "5년임대"].index
api_mh.loc[idx_5rent, '공급유형'] = "공공임대(5년)"
idx_10rent = api_mh[api_mh['공급유형'] == "10년임대"].index
api_mh.loc[idx_10rent, '공급유형'] = "공공임대(10년)"
idx_50rent = api_mh[api_mh['공급유형'] == "50년임대"].index
api_mh.loc[idx_50rent, '공급유형'] = "공공임대(50년)"

api_mh = api_mh[api_mh['공급유형'] != '매입임대']

api_mh

['국민임대' '공공임대(50년)' '영구임대' '임대상가' '공공임대(10년)' '공공임대(분납)' '장기전세' '공공분양'
 '행복주택' '공공임대(5년)']
['50년임대' '장기전세' '국민임대' '매입임대' '행복주택' '영구임대' '10년임대' '5년임대']


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,공급유형별세대수,임대보증금,임대료,단지내주차면수
0,31236970,57,아파트,서울특별시,공공임대(50년),38.1752,57,39010000,173100,534
1,31236970,57,아파트,서울특별시,공공임대(50년),40.0125,57,40800000,181000,534
2,31236970,57,아파트,서울특별시,공공임대(50년),50.0106,57,51000000,226300,534
3,30583412,168,아파트,서울특별시,공공임대(50년),33.8500,168,26019000,213600,867
4,30583336,534,아파트,서울특별시,공공임대(50년),32.7600,534,13510000,178000,263
...,...,...,...,...,...,...,...,...,...,...
13703,30699399,71,아파트,제주특별자치도,공공임대(10년),84.7200,71,0,0,306
13704,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8200,71,0,0,306
13705,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8400,71,0,0,306
13706,30699399,71,아파트,제주특별자치도,공공임대(10년),84.8500,71,0,0,306


In [ ]:
# 주차수 0
api_mh = api_mh[api_mh['주차수'] != 0]
print(api_mh['공급유형'].unique())
api_mh

KeyError: ignored

In [ ]:
import statsmodels.api as sm

fig = make_subplots(2,2, column_widths=[0.7, 0.3])
fig.update_layout(width=1000, height=1000)
train_code_total_households = []
train_code_park_nums = []
for i, code in enumerate(train['단지코드'].unique()):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    train_code_total_households.append(temp.loc[0, "총세대수"])
    train_code_park_nums.append(temp.loc[0, "단지내주차면수"])

train_regline = sm.OLS(train_code_park_nums, sm.add_constant(train_code_total_households)).fit().fittedvalues
fig.add_trace(go.Scatter(x=train_code_total_households, y=train_code_park_nums,
                         mode="markers", marker_color=px.colors.qualitative.Plotly[0], 
                         name="train"), row=1, col=1)

fig.add_trace(go.Scatter(x=train_code_total_households, y=train_regline,
                         mode="lines", marker_color="black", 
                         name="train_reg"), row=1, col=1)

fig.add_trace(go.Box(y= np.array(train_code_total_households) / np.array(train_code_park_nums),
                         marker_color=px.colors.qualitative.Plotly[0], 
                         name="train_box"), row=1, col=2)

api_mh_code_total_households = []
api_mh_code_park_nums = []
for i, code in enumerate(api_mh['단지코드'].unique()):
    temp = api_mh[api_mh['단지코드']==code]
    temp.index = range(temp.shape[0])
    api_mh_code_total_households.append(temp.loc[0, "총세대수"])
    api_mh_code_park_nums.append(temp.loc[0, "주차수"])

api_mh_regline = sm.OLS(api_mh_code_park_nums, sm.add_constant(api_mh_code_total_households)).fit().fittedvalues
fig.add_trace(go.Scatter(x=api_mh_code_total_households, y=api_mh_code_park_nums,
                         mode="markers", marker_color=px.colors.qualitative.Plotly[1], 
                         name="api_mh"), row=2, col=1)

fig.add_trace(go.Scatter(x=api_mh_code_total_households, y=api_mh_regline,
                         mode="lines", marker_color="black", 
                         name="api_mh_reg"), row=2, col=1)

fig.add_trace(go.Box(y= np.array(api_mh_code_total_households) / np.array(api_mh_code_park_nums),
                         marker_color=px.colors.qualitative.Plotly[1], 
                         name="api_mh_box"), row=2, col=2)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



KeyError: ignored

In [ ]:
api_mh = api_mh[api_mh['총세대수']/api_mh['주차수'] < 10]
api_mh

### **데이터 분포**
---


In [ ]:
numeric_cols = []
categoric_cols = []
for col in train.columns:
    if train[col].dtypes != "object":
        numeric_cols.append(col)
    else:
        categoric_cols.append(col)

numeric_cols, categoric_cols

In [ ]:
for col in categoric_cols:
    result = train[col].unique()
    result.sort()
    print(len(result))
    print(result)

In [ ]:
fig = plt.figure(figsize=(18, 18))
for i, n in enumerate(numeric_cols):
    plt.subplot(5, 2, i+1)
    plt.subplots_adjust(hspace=0.3)
    sns.histplot(train[n], color='tab:blue')
    try:
        sns.histplot(api_lh[n], color='tab:green')
        sns.histplot(api_mh[n], color='tab:orange')
    except:
        pass
    if n != "등록차량수":
        sns.histplot(test[n], color='tab:red')

In [ ]:
sns.histplot(api_mh['주차수'], color='tab:orange')

In [ ]:
api_mh_nonzero = api_mh[api_mh['주차수']!=0].copy()
fig = px.box(api_mh_nonzero, y="주차수", width=600, height=600)
fig.show()

In [ ]:
area_crit = [0, 50]
area_family_all = train['전용면적별세대수']
area_familys = {}
for i, c in enumerate(area_crit):
    if (i == len(area_crit) - 1):
        area_familys[c] = train.loc[train[(train['전용면적'] > area_crit[i])].index, '전용면적별세대수'].sum()
    else:
        area_familys[c] = train.loc[train[(train['전용면적'] > area_crit[i]) & (train['전용면적'] < area_crit[i+1])].index, '전용면적별세대수'].sum()
area_familys

In [ ]:
fig = plt.figure(figsize=(22, 22))
for i, c in enumerate(categoric_cols):
    plt.subplot(5, 2, i+1)
    plt.subplots_adjust(hspace=0.3)
    sns.histplot(train[c])

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(train.corr(), vmin=-1, vmax=1, annot=True, linewidths=0.01)
plt.show()

In [ ]:
def plot_dist_bygroup(data, group, targets, bin_num=100):
    """Plot distribution of samples which matched with corresponding
    targets and group from data.
    
    Args:
        data: Data set to use
        group: Name of the column which contains the all the targets
        targets: Name of the values which are in the data['group']

    Assertions:
        1) Group name does not match to columns in data.
        2) Any target name in targets does not match to unique members of group.

    """
    assert group in data.columns, "Unkown group input"
    assert all(target in data[group].unique() for target in targets), "Unknown target input"
    grouped = {target:None for target in targets}
    for target in targets:
        grouped[target] = data[data[group]==target]

    fig = plt.figure(figsize=(22, 44))
    for i, n in enumerate(numeric_cols):
        r_min = grouped[targets[0]][n].min()
        r_max = grouped[targets[0]][n].max()
        for target in targets:
            if grouped[target][n].min() < r_min:
                r_min = grouped[target][n].min()
            if grouped[target][n].max() > r_max:
                r_max = grouped[target][n].max()
        if grouped[targets[0]][n].dtype == int:
            r_width = np.ceil((r_max - r_min) / bin_num)
        else:
            r_width = (r_max - r_min) / bin_num
        colors = iter([plt.cm.tab20(i) for i in range(20)])
        for target in targets:
            color = next(colors)
            if r_width > sys.float_info.epsilon:
                plt.subplot(5, 2, i+1)
                plt.subplots_adjust(hspace=0.3)
                sns.histplot(grouped[target][n], color=color, label=target,
                                bins=np.arange(r_min, r_max+r_width, r_width))
                plt.legend()
            else:
                plt.subplot(5, 2, i+1)
                plt.subplots_adjust(hspace=0.3)
                sns.histplot(grouped[target][n], color=color, label=target)
                plt.legend()

plot_dist_bygroup(train, '임대건물구분', train['임대건물구분'].unique())
plot_dist_bygroup(train, '지역', train['지역'].unique())
plot_dist_bygroup(train, '공급유형', train['공급유형'].unique())
plot_dist_bygroup(train, '자격유형', train['자격유형'].unique())

In [ ]:
def plot_heatmap_bygroup(data, group, targets):
    assert group in data.columns, "Unkown group input"
    assert all(target in data[group].unique() for target in targets), "Unknown target input"
    grouped = {target:None for target in targets}
    for target in targets:
        grouped[target] = data[data[group]==target]
    
    for target in targets:
        plt.figure(figsize=(14, 10))
        heatmap = sns.heatmap(grouped[target].corr(), vmin=-1, vmax=1, annot=True, linewidths=0.01)
        heatmap.set_title("{target} in {group}".format(group=group, target=target))
        plt.show()

plot_heatmap_bygroup(train, '임대건물구분', train['임대건물구분'].unique())
plot_heatmap_bygroup(train, '지역', train['지역'].unique())
plot_heatmap_bygroup(train, '공급유형', train['공급유형'].unique())
plot_heatmap_bygroup(train, '자격유형', train['자격유형'].unique())

### **단지 통합**

In [ ]:
tmp_train = train.copy()
tmp_test = test.copy()

In [ ]:
tmp_train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

tmp_test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [ ]:
tmp_train['전용면적'] = tmp_train['전용면적']//5*5
tmp_test['전용면적'] = tmp_test['전용면적']//5*5

idx = tmp_train[train['전용면적']>100].index
tmp_train.loc[idx, '전용면적'] = 100
idx = tmp_test[test['전용면적']>100].index
tmp_test.loc[idx, '전용면적'] = 100

idx = tmp_train[train['전용면적']<15].index
tmp_train.loc[idx, '전용면적'] = 15
idx = tmp_test[test['전용면적']<15].index
tmp_test.loc[idx, '전용면적'] = 15

In [ ]:
tmp_test

In [ ]:
fig = go.Figure(data=[go.Histogram(x=tmp_train['임대보증금'], cumulative_enabled=True,  histnorm='percent', nbinsx=100)])
fig.show()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=tmp_train['임대료'], cumulative_enabled=True,  histnorm='percent', nbinsx=100)])
fig.show()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=tmp_train['전용면적'], cumulative_enabled=True,  histnorm='percent', nbinsx=100)])
fig.show()

In [ ]:
columns = ['단지코드', '총세대수', '공가율', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수' 
areas = [0.0, 36.0, 45.0, 50.0]
area_columns = ['면적_' + str(a) for a in areas]
area_columns

In [ ]:
### 단지코드, 총세대수, 법정주차대수, 지역, 공급유형(최대), 공가율, 자격유형(최대), 임대보증금, 임대료, 주차면수, 등록차량수
### 임대건물, 전용면적, 전용면적별 세대수 > 법정주차대수 계산
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
def get_legal_count(data):
    count = pd.DataFrame()
    for idx, row in data.iterrows():
        if row['임대건물구분'] == '아파트':
            if row['지역'] == 14:
                count.loc[idx, '법정주차대수'] = row['전용면적'] * row['전용면적별세대수'] * 1.0/75
            elif row['지역'] in [1, 2, 7, 9, 13, 15]:
                count.loc[idx, '법정주차대수'] = row['전용면적'] * row['전용면적별세대수'] * 1.0/85
            else:
                count.loc[idx, '법정주차대수'] = row['전용면적'] * row['전용면적별세대수'] * 1.0/95
        else:
            count.loc[idx, '법정주차대수'] = row['전용면적'] * row['전용면적별세대수'] * 1.0/200
    return count.sum().values[0]

for i, code in tqdm(enumerate(tmp_train['단지코드'].unique())):
    temp = tmp_train[tmp_train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for j, col in enumerate(areas):
        if (j == len(areas)-1):
            new_train.loc[i, area_columns[j]] = temp[temp['전용면적']>= col]['전용면적별세대수'].sum()
        else:
            new_train.loc[i, area_columns[j]] = temp[(temp['전용면적']>= col) & (temp['전용면적']< areas[j+1])]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(tmp_test['단지코드'].unique())):
    temp = tmp_test[tmp_test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for j, col in enumerate(areas):
        if (j == len(areas)-1):
            new_test.loc[i, area_columns[j]] = temp[temp['전용면적']>= col]['전용면적별세대수'].sum()
        else:
            new_test.loc[i, area_columns[j]] = temp[(temp['전용면적']>= col) & (temp['전용면적']< areas[j+1])]['전용면적별세대수'].sum()

In [ ]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

In [ ]:
new_train

In [ ]:
new_test

### **학습 및 추론**
--- 


In [ ]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, 
                                                      random_state=42, shuffle=True)

In [ ]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(
        loss_function='MAE',
        n_estimators=300, 
        learning_rate=0.05, 
        random_state=42     
    )
cat.fit(x_train, y_train, eval_set=(x_valid, y_valid), use_best_model=True)

In [ ]:
from sklearn.metrics import mean_absolute_error

score = mean_absolute_error(y_valid, cat.predict(x_valid))
score

In [ ]:
pred = cat.predict(x_test)

In [ ]:
submission['num'] = pred

In [ ]:
submission.to_csv(result_path + 'cat_base.csv', index=False)